## Laboratorio 3 Interrupciones - Sistemas Operativos
#### Por: Ricardo Mendoza

#### 1. Utilización de un sistema operativo de direccionamiento de 16 bits

Para este Laboratorio se hara uso de una maquina virtual de un sistema de direccionamiento de 16 bits ( tamaño de registros de 32 bits). Dicha maquina virtual contiene el compilar Turbo C version 3 la cual utilizaremos para los siguientes ejercicios de codigo en C++.

#### 2. Explicación del vector de interrupciones y los tipos de interrupciones


Partiendo de que una interrupcion es una señal emitida por un programa o por un dispositivo de la computadora para que el CPU se detenga de hacer lo que hace y empiece a ejecutar las instrucciones que se almacenan en una direccion especifica de la memoria, dicha direccion especifica de memoria es proveida por el vector de interrupciones. Analizando un poco como seria el flujo de una interrupcion quedaria de la siguiente forma: Primer punto, el CPU esta trabajando en un proceso X; segundo punto, un dispositivo recibe un input entonces envia una señal o una interrupción al cpu con la intención de ser atendido; tercer punto, el CPU realiza un cambio de contexto, almacenando las direcciones y registros del proceso X para luego ser restablecido; cuarto punto , la interrupcion contiene parametros para indicar que interrupcion se requiere, en base a estos parametros es posible buscar la dirección de la rutina de ejecucion de dicha interrupcion en un vector que contiene las direcciones de la primera instrucción a ser ejecutada por la rutina de cada interrupción.


Las Interrupciones pueden ser de 3 tipos:
* Interrupciones de Hardware

    Estas interrupciones no se san por ninguna instruccion de algun programa que se esta ejecutando, sino se dan exclusivamente por los dispositivos de entrada o salida como por ejemplo el teclado, el mouse, entre otros.
    

* Excepciones

    Estas son interrupciones que se generan por un error, normalmente por operaciones invalidas o por el acceso a segmentos de memoria no permitidos. Esta interrupcion genera un cambio de contexto a los procesos del sistema operativo para que este maneje el error.
    
* Interrupciones por Software

    Estos son los famosos syscalls o system calls, dichas interrupciones son generadas por un programa en tiempo de ejecucion. Algunos ejemplos de estas interrupciones son cuando un programa requiere un servicio del sistema operativo para escribir sobre un archivo o cuando el programa necesita obtener la hora de la maquina.
